# BranchPro: inference of R_t with Poisson Binomial Noise

The first part of the notebook includes a forward simulation of the incidence numbers for an example branching process model with Poisson Binomial noise.

The second part of the notebook focuses on the computation of the posterior of the reproduction number for the inference using the data from the previous section, using two methods:
- using the posterior class implementation.

The mean and 95% interval quantiles are plotted together to illustrate a sensible trajectory of the R profile in time for both methods of inference.

In [1]:
# Import libraries
import numpy as np
import math
import branchpro
import scipy.stats
from branchpro.apps import ReproductionNumberPlot
import plotly.graph_objects as go
import pandas as pd
from cmdstanpy import CmdStanModel, cmdstan_path
import arviz as az
import nest_asyncio
nest_asyncio.apply()

num_timepoints = 100 # number of days for incidence data


/Users/ioaros/opt/anaconda3/envs/testbranchpro2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Parameterize example branching process model

In [2]:
# Build the next generation time distribution theta_s
ws_mean = 8
ws_std = 2
w_dist = scipy.stats.norm(ws_mean, scale=ws_std).pdf(np.arange(1, 21))
disc_w = 0.02 * w_dist / np.max(w_dist)

# Simulate incidence data
initial_mu = 3 * 7
next_gen = disc_w
m = branchpro.PoiBinBranchProModel(initial_mu, next_gen)
new_mus = [21, 14, 7]
start_times = [0, 20, 40]
m.set_mean_contact(new_mus, start_times)
parameters = 10 # initial number of cases
times = np.arange(num_timepoints)
 
cases = m.simulate(parameters, times)
print(cases)

[10.  0.  0.  1.  0.  0.  2.  2.  5.  2.  4.  1.  2.  2.  4.  4.  9.  6.
  4.  7.  9.  3.  6.  7.  7.  9. 12.  8. 12. 10.  8. 10. 13. 13. 16. 13.
 20. 12. 14. 19. 14. 15.  9.  9. 16. 11.  6. 11. 12.  5. 12. 10.  9.  8.
  4. 11.  7. 11.  5.  5.  3.  4.  5.  5.  5.  7.  7.  4.  6.  3.  1.  3.
  2.  2.  3.  2.  5.  6.  2.  1.  1.  0.  1.  2.  3.  0.  2.  1.  1.  0.
  1.  0.  1.  2.  0.  0.  1.  0.  1.  1.]


## Plot local incidence numbers

In [3]:
# Plot (bar chart cases each day)
fig = go.Figure()

# Plot of incidences
fig.add_trace(
    go.Bar(
        x=times,
        y=cases,
        name='Incidences'
    )
)

# Add axis labels
fig.update_layout(
    xaxis_title='Time (days)',
    yaxis_title='New cases'
)

fig.show()

## Compute the posterior distribution using STAN

In [4]:
poibin_model = CmdStanModel(stan_file=os.path.join(
        '../branchpro/stan_models/poibin.stan'))

poibin_data = {
    'N': num_timepoints,
    'S': len(next_gen),
    'I': cases.astype(np.integer).tolist(),
    'Theta': next_gen.tolist()}

16:34:07 - cmdstanpy - INFO - compiling stan file /var/folders/ph/jyxnc9y52svgq2k5lt2q4r000000gp/T/tmphoba_930/tmpt3s8zwrm.stan to exe file /Users/ioaros/Desktop/Software Project/branchpro/branchpro/stan_models/poibin


ValueError: Failed to compile Stan model '/Users/ioaros/Desktop/Software Project/branchpro/branchpro/stan_models/poibin.stan'. Console:

--- Translating Stan model to C++ code ---
bin/stanc --filename-in-msg=poibin.stan --o=/var/folders/ph/jyxnc9y52svgq2k5lt2q4r000000gp/T/tmphoba_930/tmpt3s8zwrm.hpp /var/folders/ph/jyxnc9y52svgq2k5lt2q4r000000gp/T/tmphoba_930/tmpt3s8zwrm.stan
Syntax error in 'poibin.stan', line 49, column 8 to column 10, parsing error:
   -------------------------------------------------
    47:          return alpha[N + 1, y + 1];
    48:      }
    49:      int [] evaluate_previous_cases (int t, array [] int aI, array [] real aMu) {
                 ^
    50:          array [t-1] int rev_prev_cases;
    51:          int counter;
   -------------------------------------------------

An identifier is expected after the type as a function argument name.
It looks like you are trying to use the old array syntax.
Please use the new syntax: 
array[] int
make: *** [/var/fold

In [ ]:
fit = poibin_model.sample(
    data=poibin_data, seed=10, chains=3, iter_sampling=1000
    )

samples = az.from_cmdstanpy(fit)

az.summary(samples)

Building...



Building: found in cache, done.Messages from stanc:
Warning in '/var/folders/ph/jyxnc9y52svgq2k5lt2q4r000000gp/T/httpstan_wzxm3eiu/model_lg4yjckc.stan', line 50, column 4: Declaration
    of arrays by placing brackets after a type is deprecated and will be
    removed in Stan 2.33.0. Instead use the array keyword before the type.
    This can be changed automatically using the auto-format flag to stanc
Warning in '/var/folders/ph/jyxnc9y52svgq2k5lt2q4r000000gp/T/httpstan_wzxm3eiu/model_lg4yjckc.stan', line 111, column 22: Argument
    14 suggests there may be parameters that are not unit scale; consider
    rescaling with a multiplier (see manual section 22.12).
Warning in '/var/folders/ph/jyxnc9y52svgq2k5lt2q4r000000gp/T/httpstan_wzxm3eiu/model_lg4yjckc.stan', line 102, column 8: A
    control flow statement depends on parameter(s): Mu.
Warning in '/var/folders/ph/jyxnc9y52svgq2k5lt2q4r000000gp/T/httpstan_wzxm3eiu/model_lg4yjckc.stan', line 76, column 12: A
    control flow statement

In [ ]:
az.rcParams['plot.max_subplots'] = 2 * num_timepoints

az.plot_trace(
    samples,
    var_names=('Mu'),
    filter_vars='like',
    compact=False)

In [ ]:
# Eliminate burn-in iterations (1/2 of the chain lengths)
chain_samples = fit.draws()[500:, :, 7:]

# Evaluate the model for all parameter sets in the samples
n_param, n_sample, n_chains = chain_samples.shape

extended_samples = np.concatenate((
    chain_samples[:, 0, :],
    chain_samples[:, 1, :],
    chain_samples[:, 2, :]), axis=0)

thinning = max(1, int(n_sample * n_chains / 500))

new_intervals = pd.DataFrame({
    'Time Points': np.arange(R_t_start, num_timepoints),
    'Mean': np.mean(extended_samples[::thinning, :], axis=0),
    'Lower bound CI': np.quantile(extended_samples[::thinning, :], 0.025, axis=0),
    'Upper bound CI': np.quantile(extended_samples[::thinning, :], 0.975, axis=0),
    'Central Probability': (L1+L2) * [0.95]
})

## Inference plot using class method results

In [ ]:
fig = ReproductionNumberPlot()

fig.add_ground_truth_rt(ground_truth)
fig.add_interval_rt(new_intervals)

fig.update_labels(time_label='Time (Day)', r_label='R_t')

fig.show_figure()

/Users/ioaros/Desktop/Software Project/branchpro/branchpro/apps/_reproduction_number_plot.py:27: UserWarning:

Labels do not match. They will be updated.

